In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
import mercantile
from tqdm import tqdm
import os
import tempfile
import fiona

## Step 1 - Define our area of interest (AOI)

We define our area of interest (or AOI) as a GeoJSON geometry, then use the `shapely` library to get the bounding box.

**Note**: the coordinate reference system for the GeoJSON should be "EPSG:4326", i.e. in global lat/lon format.

In [2]:
# Geometry copied from https://geojson.io
aoi_geom = {
    "coordinates": [
        [
            [25.664063, 42.111264],
            [26.173827, 36.034885],
            [36.017578, 35.603719],
            [44.938477, 36.774092],
            [44.890137, 39.609074],
            [42.934570, 41.718851],
            [35.055176, 42.439674],
            [30.212402, 42.143856],
            [25.976073, 42.195969],
            [25.664063, 39.388660],
        ]
    ],
    "type": "Polygon",
}
aoi_shape = shapely.geometry.shape(aoi_geom)
minx, miny, maxx, maxy = aoi_shape.bounds

output_fn = "example_building_footprints.geojson"

## Step 2 - Determine which tiles intersect our AOI

In [3]:
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(int(mercantile.quadkey(tile)))
quad_keys = list(quad_keys)
print(f"The input area spans {len(quad_keys)} tiles: {quad_keys}")

The input area spans 364 tiles: [122112000, 122112001, 122112002, 122112003, 122112010, 122112011, 122112012, 122112013, 120332302, 120332303, 120332312, 120332313, 120332320, 120332321, 120332322, 120332323, 120332330, 120332331, 120332332, 120332333, 122110000, 122110001, 122110002, 122110003, 122110010, 122110011, 122110012, 122110013, 122110020, 122110021, 122110022, 122110023, 122110030, 122110031, 122110032, 122110033, 122112100, 122112101, 122112102, 122112103, 122112110, 122112111, 122112112, 122112113, 122110100, 122110101, 122110102, 122110103, 122110110, 122110111, 122110112, 122110113, 122110120, 122110121, 122110122, 122110123, 122110130, 122110131, 122110132, 122110133, 122110200, 122110201, 122110202, 122110203, 120322302, 120322303, 122110210, 122110211, 122110212, 122110213, 120322312, 120322313, 122110220, 122110221, 122110222, 122110223, 120322320, 120322321, 120322322, 120322323, 122110230, 122110231, 122110232, 122110233, 120322330, 120322331, 120322332, 120322333,

## Step 3 - Download the building footprints for each tile that intersects our AOI and crop the results

This is where most of the magic happens. We download all the building footprints for each tile that intersects our AOI, then only keep the footprints that are _contained_ by our AOI.

*Note*: this step might take awhile depending on how many tiles your AOI covers and how many buildings footprints are in those tiles.

In [4]:
df = pd.read_csv(
    "https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv"
)

idx = 0
combined_rows = []

with tempfile.TemporaryDirectory() as tmpdir:
    # Download the GeoJSON files for each tile that intersects the input geometry
    tmp_fns = []
    for quad_key in tqdm(quad_keys):
        rows = df[df["QuadKey"] == quad_key]
        if rows.shape[0] == 1:
            url = rows.iloc[0]["Url"]

            df2 = pd.read_json(url, lines=True)
            df2["geometry"] = df2["geometry"].apply(shapely.geometry.shape)

            gdf = gpd.GeoDataFrame(df2, crs=4326)
            fn = os.path.join(tmpdir, f"{quad_key}.geojson")
            tmp_fns.append(fn)
            if not os.path.exists(fn):
                gdf.to_file(fn, driver="GeoJSON")
        elif rows.shape[0] > 1:
            raise ValueError(f"Multiple rows found for QuadKey: {quad_key}")
        else:
            raise ValueError(f"QuadKey not found in dataset: {quad_key}")

    # Merge the GeoJSON files into a single file
    for fn in tmp_fns:
        with fiona.open(fn, "r") as f:
            for row in tqdm(f):
                row = dict(row)
                shape = shapely.geometry.shape(row["geometry"])

                if aoi_shape.contains(shape):
                    if "id" in row:
                        del row["id"]
                    row["properties"] = {"id": idx}
                    idx += 1
                    combined_rows.append(row)

  0%|          | 1/364 [00:13<1:22:53, 13.70s/it]


ValueError: QuadKey not found in dataset: 122112001

## Step 4 - Save the resulting footprints to file

In [ ]:
schema = {"geometry": "Polygon", "properties": {"id": "int"}}

with fiona.open(output_fn, "w", driver="GeoJSON", crs="EPSG:4326", schema=schema) as f:
    f.writerecords(combined_rows)